In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import pyodbc
from sqlalchemy import create_engine
import urllib

In [ ]:
pd.options.display.min_rows = 100

### need to do each report separtely. for delinq - if one line item is over 270 days then need to mark in csv file that all the line items for that ar account also have 1 for delinq for when it is merged later on

In [ ]:
file = "kind_origin_cannabis_ar"

In [ ]:
# ar = pd.read_csv('../ar_reports/orig/desert_road_ar_02_29_2020.csv')

In [ ]:
ar = pd.read_csv(f'../ar_reports/orig/{file}.csv')

In [ ]:
ar

In [ ]:
ar.dtypes

In [ ]:
ar.info()

In [ ]:
ar.head(2)

In [ ]:
ar.shape

In [ ]:
# ar[ar['license_number'].isnull()]

In [ ]:
# ar_final = ar[~ar['state_license'].isnull()]

In [ ]:
# ar_final = ar

In [ ]:
ar_final = ar[ar['license_number'] != '0']

In [ ]:
# This should fix the negative sum of dpd in the combined list
days_over_due = []
for days in ar_final['days_over_due']:
    if days < 0:
        days = 0
    days_over_due.append(days)
ar_final['days_over_due'] = days_over_due

In [ ]:
# list2 = ar_final[ar_final['bal_out'] < 0].index
# len(list2)

In [ ]:
# pd.set_option('mode.chained_assignment', None)

# ar_final['orig_amt'].loc[list2] = 0
# ar_final['bal_out'].loc[list2] = 0
# ar_final['current'].loc[list2] = 0
# ar_final['30_days'].loc[list2] = 0
# ar_final['60_days'].loc[list2] = 0
# ar_final['90_days'].loc[list2] = 0
# ar_final['over_90_days'].loc[list2] = 0

In [ ]:
# Idk why the above code was missing some negatives, but this clears that up

for col in ['current', '30_days', '60_days', '90_days', 'over_90_days']:
    cleaned_data = []
    for num in ar_final[col]:
        if num < 0:
            num = 0
        cleaned_data.append(num)
    ar_final[col] = cleaned_data
    
for n in range(len(ar_final)):
    if ar_final['orig_amt'].iloc[n] < 0:
        ar_final['orig_amt'].iloc[n] = 0

In [ ]:
ar_final

In [ ]:
count_payor = ar_final.groupby(['ar_account', 'date'], as_index=False)['license_number'].count()
count_payor.rename(columns={'license_number': 'count_payor'}, inplace=True)

In [ ]:
count_payor

In [ ]:
sum_days_over = ar_final.groupby(['ar_account', 'date'], as_index=False)['days_over_due'].sum()
sum_orig_amt_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['orig_amt'].sum()
sum_bal_out_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['bal_out'].sum()
sum_current_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['current'].sum()
sum_30_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['30_days'].sum()
sum_60_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['60_days'].sum()
sum_90_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['90_days'].sum()
sum_over_90_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['over_90_days'].sum()

In [ ]:
# Wrote this before I realized I sould just fix the one two below
# count_payor.merge(sum_days_over).merge(sum_orig_amt_clean).merge(sum_bal_out_clean).merge(sum_current_clean).merge(sum_30_day_clean).merge(sum_60_day_clean).merge(sum_90_day_clean).merge(sum_over_90_day_clean)

In [ ]:
count_payor

In [ ]:
data_frames = [count_payor, sum_days_over, sum_orig_amt_clean, sum_bal_out_clean, sum_current_clean, sum_30_day_clean,
               sum_60_day_clean, sum_90_day_clean, sum_over_90_day_clean]

In [ ]:
df_sums = reduce(lambda left,right: pd.merge(left,right,on=['ar_account', 'date'], how='outer'), data_frames)

In [ ]:
df_sums.head()

In [ ]:
ar_final

In [ ]:
ar_final['delinquent'] = 0

In [ ]:
ar_to_merge = ar_final[['license_number', 'license_provided', 'ar_account', 'date', 'client', 'summary','delinquent']]

In [ ]:
pd.set_option('mode.chained_assignment', None)

ar_to_merge.drop_duplicates(subset='ar_account', keep='first', inplace=True)

In [ ]:
df_sums = pd.merge(df_sums, ar_to_merge.drop(columns={'date'}), on=['ar_account'], how='outer')

In [ ]:
df_sums.rename(columns={'days_over_due': 'sum_days_over'}, inplace=True)

In [ ]:
df_sums

In [ ]:
df_final = df_sums[['license_number', 
          'license_provided',
          'ar_account', 
          'count_payor', 
          'sum_days_over',
           'orig_amt', 
          'bal_out', 
          'current', 
          '30_days', 
          '60_days', 
          '90_days',
           'over_90_days', 
          'delinquent', 
          'date', 
          'client', 
          'summary'
       ]]

In [ ]:
df_final

In [ ]:
df_final[df_final['delinquent'] == 1]

In [ ]:
df_final

In [ ]:
# This code should remove the negatives that were previousloy missed

df_final['not_neg'] = df_final['bal_out'] > 0

df_final

df_not_neg = df_final[df_final['not_neg'] == True]
df_not_neg = df_not_neg.reset_index().drop(columns={'not_neg', 'index'})

In [ ]:
df_not_neg

In [ ]:
df_not_neg.to_csv(f'../ar_reports/orig/no_neg/{file}_no_neg.csv', index=False)

In [ ]:
# df_final.to_csv(f'../ar_reports/orig/no_neg/{file}_no_neg.csv', index=False)